In [2]:
import pandas as pd
df = pd.read_csv("./LondonBikeJourneyAug2023.csv")

In [3]:
# 数据预处理

# 1. 删除无用列
df = df.drop(["Number", "Bike model", "Total duration", "Start station number", "End station number"], axis=1)

# 2. 检查并删除重复行
df = df.drop_duplicates()

# 3. 检查并删除缺失值
df = df.dropna()

# 4. 处理空白字符
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [4]:
# 数据转换和规范化

# 1. 将 Start date 和 End date 转换为 datetime 格式
df['Start date'] = pd.to_datetime(df['Start date'])
df['End date'] = pd.to_datetime(df['End date'])

# 2. 将 Total duration 从毫秒转换为分钟, 保留两位小数
df['Total duration (min)'] = df['Total duration (ms)'] / 60000

In [5]:
df.head()

,Start date,Start station,End date,End station,Bike number,Total duration (ms),Total duration (min)
0,2023-08-01 00:00:00,"Kennington Lane Rail Bridge, Vauxhall",2023-08-01 00:17:00,"Albert Embankment, Vauxhall",23715,1006663,16.777717
1,2023-08-01 00:00:00,"Kennington Lane Rail Bridge, Vauxhall",2023-08-01 00:17:00,"Albert Embankment, Vauxhall",41267,1007128,16.785467
2,2023-08-01 00:00:00,"Euston Road, Euston",2023-08-01 00:11:00,"Baldwin Street, St. Luke's",53180,666395,11.106583
3,2023-08-01 00:01:00,"Old Brompton Road, South Kensington",2023-08-01 00:12:00,"Grosvenor Road, Pimlico",53431,713059,11.884317
4,2023-08-01 00:01:00,"Lower Marsh, Waterloo",2023-08-01 00:17:00,"Vauxhall Walk, Vauxhall",54758,963786,16.063100


In [6]:
# 分析每天的总骑行次数及平均骑行时间 (分钟)
day_df = df.groupby(df['Start date'].dt.date).size().reset_index(name='Counts')
day_df['Average duration (min)'] = df.groupby(df['Start date'].dt.date)['Total duration (min)'].mean().values.round(2)

# 重命名列
day_df = day_df.rename(columns={'Start date': 'Date'})

# 保存结果
day_df.to_csv("./per_day_data.csv", index=False)

day_df.head()

,Date,Counts,Average duration (min)
0,2023-08-01,30055,23.71
1,2023-08-02,20622,19.67
2,2023-08-03,29822,23.14
3,2023-08-04,24950,27.17
4,2023-08-05,11638,29.56


In [7]:
# 分析前四周每周的总骑行次数
week_df = df['Start date'].dt.isocalendar()
week_df = week_df.groupby(week_df['week']).size().reset_index(name='Counts')

# 去掉第一周和最后一周
week_df = week_df.drop([0, 4])

# 相较于前一周的增长率, 初始值为 0
week_df['Growth rate'] = week_df['Counts'].pct_change().fillna(0).map(lambda x: format(x, '.2%'))

# 重命名列
week_df = week_df.rename(columns={'week': 'Week'})

# 保存结果
week_df.to_csv("./per_week_data.csv", index=False)

week_df.head()

,Week,Counts,Growth rate
1,32,180141,0.00%
2,33,184177,2.24%
3,34,174292,-5.37%


In [8]:
# 分析每周的总骑行次数及平均骑行时间 (分钟)
weed_day_df = df.groupby(df['Start date'].dt.day_of_week).size().reset_index(name='Counts')
weed_day_df['Average duration (min)'] = df.groupby(df['Start date'].dt.day_of_week)['Total duration (min)'].mean().values.round(2)

# 重命名列
weed_day_df = weed_day_df.rename(columns={'Start date': 'Day of week'})

# 保存结果
weed_day_df.to_csv("./per_weekday_data.csv", index=False)

weed_day_df.head()

,Day of week,Counts,Average duration (min)
0,0,98141,25.66
1,1,138650,23.88
2,2,143337,23.07
3,3,141325,22.55
4,4,97157,25.20


In [9]:
# 以小时为单位分析每天的骑行次数
hour_df = df.groupby(df['Start date'].dt.hour).size().reset_index(name='Counts')

# 重命名列
hour_df = hour_df.rename(columns={'Start date': 'Hour'})

# 保存结果
hour_df.to_csv("./per_hour_data.csv", index=False)

hour_df.head()

,Hour,Counts
0,0,7494
1,1,4465
2,2,2844
3,3,1897
4,4,2053


In [15]:
# 分析以州为单位，分别计算为起点和终点的总骑行次数，平均骑行时长,放到一张表中
start_df = df.groupby('Start station').agg({'Start station': 'count', 'Total duration (min)': 'mean'}).rename(columns={'Start station': 'Counts', 'Total duration (min)': 'Average Duration (min)'}).reset_index()
end_df = df.groupby('End station').size().reset_index(name='Counts')

# 重命名列
start_df = start_df.rename(columns={'Start station': 'Station'})
end_df = end_df.rename(columns={'End station': 'Station'})

# 合并两个表, 使用外连接，缺失值填充为 0
station_df = pd.merge(start_df, end_df, on='Station', how='outer').fillna(0).astype({'Counts_x': 'int32', 'Counts_y': 'int32'}).rename(columns={'Counts_x': 'Start Station Counts', 'Counts_y': 'End Station Counts'})

# 根据计算平均骑行次数
station_df['Average Counts'] = (station_df['Start Station Counts'] + station_df['End Station Counts']) / 2  

# 调整排序
station_df['Average Duration (min)'] = station_df['Average Duration (min)'].round(2)
station_df = station_df[['Station', 'Start Station Counts', 'End Station Counts', 'Average Counts','Average Duration (min)']]

# 保存结果
station_df.to_csv("./station_data.csv", index=False)

start_df.head()

station_df.head()

,Station,Start Station Counts,End Station Counts,Average Counts,Average Duration (min)
0,"Abbey Orchard Street, Westminster",1164,1304,1234.0,21.72
1,"Abbotsbury Road, Holland Park",335,294,314.5,17.86
2,"Aberdeen Place, St. John's Wood",727,581,654.0,17.11
3,"Aberfeldy Street, Poplar",362,371,366.5,24.34
4,"Abingdon Green, Westminster",826,1069,947.5,24.10


In [11]:
# 以起点和终点相同的订单为一组，分别计算总骑行次数和平均骑行时长
trip_df = df.groupby(['Start station', 'End station']).size().reset_index(name='Counts')
trip_df['Average Duration (min)'] = df.groupby(['Start station', 'End station'])['Total duration (min)'].mean().values

# 舍弃 Counts 小于 100 的行
trip_df = trip_df[trip_df['Counts'] >= 100]

# 根据 Counts 排序
trip_df = trip_df.sort_values(by='Counts', ascending=False)

# 保留两位小数
trip_df['Average Duration (min)'] = trip_df['Average Duration (min)'].round(2)

# 保存结果
trip_df.to_csv("./trip_data.csv", index=False)

trip_df.head()

,Start station,End station,Counts,Average Duration (min)
89805,"Hyde Park Corner, Hyde Park","Hyde Park Corner, Hyde Park",2966,40.90
128641,"Podium, Queen Elizabeth Olympic Park","Podium, Queen Elizabeth Olympic Park",1813,47.41
170384,"Triangle Car Park, Hyde Park","Triangle Car Park, Hyde Park",1391,46.87
18530,"Black Lion Gate, Kensington Gardens","Black Lion Gate, Kensington Gardens",1343,42.82
2926,"Albert Gate, Hyde Park","Albert Gate, Hyde Park",1227,38.35


In [12]:
# 根据 Bike number 分组，计算每辆自行车的总骑行次数
bike_df = df.groupby('Bike number').size().reset_index(name='Counts')

# 保留前 100 个数据
bike_df = bike_df.sort_values(by='Counts', ascending=False).head(100)

# 保存结果
bike_df.to_csv("./bike_data.csv", index=False)

bike_df.head()

,Bike number,Counts
10530,58819,208
11144,59681,197
11135,59664,193
10543,58836,191
11163,59712,188
